In [4]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division
from mpl_toolkits.mplot3d import Axes3D

In [137]:
sigma = 10
r = 20.96
b = 8/3

In [138]:
def lorentz_update(tn,x):
    return np.array([sigma*(x[:,1]-x[:,0]),
            -x[:,0]*x[:,2]+r*x[:,0]-x[:,1],
            x[:,0]*x[:,1]-b*x[:,2]]).T

In [139]:
def merson(tn,x,function,h):
    k1 = h*function(tn,x)
    k2 = h*function(tn+h/3,x+k1/3)
    k3 = h*function(tn+h/3,x+(k1+k2)/6)
    k4 = h*function(tn+h/2,x+(k1+k3*3)/8)
    k5 = h*function(tn+h,x+k1/2-3*k3/2+2*k4)
    return k1/6+2*k4/3+k5/6

In [141]:
def lorentz(start_pos, update_function, step = 0.01, filename = None):
    import matplotlib.animation as animation
    
    global state
    start_pos = np.asarray(start_pos,dtype=float)
    
    state = np.zeros(shape = (1001,)+start_pos.shape)
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d')
    ax.grid()
    ax.set_xlim3d(-30+min(start_pos[:,0]),max(start_pos[:,0])+30)
    ax.set_ylim3d(-30+min(start_pos[:,1]),max(start_pos[:,1])+30)
    ax.set_zlim3d(-30+min(start_pos[:,2]),max(start_pos[:,2])+30)

    string = 'Uklad lorentza \n sigma = %.1f, b = %.1f, r = %.1f, \n Bartosz Podrygajlo' % (sigma,b,r) 
    text = ax.text(0.05, 0.9, 0,string, transform=ax.transAxes)
    
    points, = ax.plot([],[], linestyle='None', marker='.', markersize = 10.)
    
    hist = []
    for i in range(start_pos.shape[0]):
        hist.append(ax.plot([],[],linestyle='-')[0])
    
    state[0] = start_pos
    
    def init():
        
        return None
    
    def animate(t):
        global state
        
        state[t+1] = state[t]  + merson(tn=t*step,
                                    x=state[t],
                                    function=update_function,
                                    h=step)
        ax.view_init(30, t)
        
        
        points._verts3d = state[t,:,0], state[t,:,1], state[t,:,2]
        
        for i in range(start_pos.shape[0]):
            hist[i]._verts3d = state[0:t,i,0],state[0:t,i,1],state[0:t,i,2]
        
        return None
    
    ani = animation.FuncAnimation(fig, animate, np.arange(1000),
                              interval=1000*step, blit=False, init_func=init)
    
    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [142]:
lorentz(start_pos=[[1,1,1],[30,30,30],[-30,-30,30],[20,20,20]],
        update_function=lorentz_update,
       filename='lorentz_1')

In [143]:
sigma = 50
r = 50.96
b = 4

In [ ]:
lorentz(start_pos=[[1,1,1],[30,30,30],[-30,-30,30],[20,20,20]],
        update_function=lorentz_update,
       filename='lorentz_2')